In [2]:
import glob
import os
import shutil
import numpy as np
import cv2

In [44]:
#フォルダリストを取得
path='kill_me_baby_datasets'
train_list=glob.glob(path+'/train/*')
#testフォルダvalフォルダを作成
for i in train_list:
    os.makedirs(i.replace('train','test'))
    os.makedirs(i.replace('train','val'))

In [45]:
for file in train_list:
    #フォルダ内画像のパスを取得
    pics = glob.glob(file+'/*.png')
    pick_num = len(pics)//10
    if not pick_num: continue
    #20%を選び出す
    choice = np.random.choice(pics, pick_num*2, replace=False)
    #選んだ画像をtestとvalフォルダに移動
    for test_pic in choice[:pick_num]:
        shutil.move(test_pic, test_pic.replace('train','test'))
    for val_pic in choice[pick_num:]:
        shutil.move(val_pic, val_pic.replace('train','val'))

In [49]:
import keras

In [55]:
def get_data(dpath):
    path_list=glob.glob(dpath+'/*')
    #画像ファイル、ラベルを入れる箱を用意
    x=[]
    y=[]

    #フォルダ名でループを回す
    for label,pic_path in enumerate(path_list):
        #サブフォルダ内のファイルリストを取得
        train_pic_list=glob.glob(pic_path+'/*.png')
        print(pic_path, len(train_pic_list))
        #画像ファイルを取得＋リサイズ、ラベルを用意
        x += [cv2.resize(cv2.imread(i), (128,128)) for i in train_pic_list]
        y += [label]*len(train_pic_list)

    #画像をfloat32に変換正規化、ラベルをカテゴリカル化
    x = np.float32(x)/255
    y = keras.utils.to_categorical(y, len(train_list))
    return x, y

In [77]:
x_train, y_train = get_data(path+'/train')
x_val, y_val = get_data(path+'/val')
x_test, y_test = get_data(path+'/test')

kill_me_baby_datasets/train\agiri 36
kill_me_baby_datasets/train\botsu 8
kill_me_baby_datasets/train\others 52
kill_me_baby_datasets/train\sonya 141
kill_me_baby_datasets/train\yasuna 273
kill_me_baby_datasets/train\yasuna_agiri 4
kill_me_baby_datasets/train\yasuna_sonya 39
kill_me_baby_datasets/train\yasuna_sonya_agiri 1
kill_me_baby_datasets/val\agiri 4
kill_me_baby_datasets/val\botsu 1
kill_me_baby_datasets/val\others 6
kill_me_baby_datasets/val\sonya 17
kill_me_baby_datasets/val\yasuna 34
kill_me_baby_datasets/val\yasuna_agiri 0
kill_me_baby_datasets/val\yasuna_sonya 4
kill_me_baby_datasets/val\yasuna_sonya_agiri 0
kill_me_baby_datasets/test\agiri 4
kill_me_baby_datasets/test\botsu 1
kill_me_baby_datasets/test\others 6
kill_me_baby_datasets/test\sonya 17
kill_me_baby_datasets/test\yasuna 34
kill_me_baby_datasets/test\yasuna_agiri 0
kill_me_baby_datasets/test\yasuna_sonya 4
kill_me_baby_datasets/test\yasuna_sonya_agiri 0


In [78]:
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape,y_test.shape)

(554, 128, 128, 3) (554, 8)
(66, 128, 128, 3) (66, 8)
(66, 128, 128, 3) (66, 8)


In [63]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam

In [65]:
input_shape = (128, 128, 3)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(GlobalAveragePooling2D())

model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 128, 128, 32)      896       
_________________________________________________________________
batch_normalization_31 (Batc (None, 128, 128, 32)      128       
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 128, 128, 32)      9248      
_________________________________________________________________
batch_normalization_32 (Batc (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 64, 64, 64)        18496     
_________________________________________________________________
batch_normalization_33 (Batc (None, 64, 64, 64)        256       
__________

In [66]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
#val_lossがbestの値を出したときにモデルの重みのみ(weight_only)を保存
checkpoint_path = path+"/weights/weights_{epoch:02d}_{val_loss:.3f}_{val_acc:.3f}.h5"
print(checkpoint_path)
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=0,
                                 save_best_only=True, save_weights_only=True, mode='auto')  

#accの向上がない=学習が止まっているとみてストップする。
earlystop = EarlyStopping(monitor='acc', patience=15, verbose=1, mode='auto')
    
# CSV出力、下に重ねていく
csv_logger = CSVLogger(path+'/weights/training.csv', append=True)
    
# val_lossの向上が見られない時に学習率を減らす。
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.1**0.5, cooldown=0, patience=5,
                                   verbose=1, min_lr=0.5e-4)
call_backs = [csv_logger,checkpoint, lr_reducer]
print('use:csv_logger,checkpoint2,lr_reducer')

kill_me_baby_datasets/weights/weights_{epoch:02d}_{val_loss:.3f}_{val_acc:.3f}.h5
use:csv_logger,checkpoint2,lr_reducer


In [68]:
history = model.fit(x_train, y_train, batch_size=32, epochs=30, verbose=1, callbacks=call_backs,
                    validation_data=(x_val, y_val))

Train on 554 samples, validate on 66 samples
Epoch 1/30
554/554 [==============================] - 5s 9ms/step - loss: 1.1983 - acc: 0.6805 - val_loss: 4.1550 - val_acc: 0.5000
Epoch 2/30
554/554 [==============================] - 1s 2ms/step - loss: 0.7039 - acc: 0.8087 - val_loss: 1.3937 - val_acc: 0.8030
Epoch 3/30
554/554 [==============================] - 1s 2ms/step - loss: 0.6324 - acc: 0.7924 - val_loss: 0.5916 - val_acc: 0.8485
Epoch 4/30
554/554 [==============================] - 1s 2ms/step - loss: 0.4764 - acc: 0.8412 - val_loss: 1.8574 - val_acc: 0.7424
Epoch 5/30
554/554 [==============================] - 1s 2ms/step - loss: 0.5131 - acc: 0.8628 - val_loss: 0.3967 - val_acc: 0.8636
Epoch 6/30
554/554 [==============================] - 1s 2ms/step - loss: 0.4227 - acc: 0.8520 - val_loss: 1.5476 - val_acc: 0.7424
Epoch 7/30
554/554 [==============================] - 1s 2ms/step - loss: 0.4185 - acc: 0.8755 - val_loss: 2.0366 - val_acc: 0.6970
Epoch 8/30
554/554 [===========

In [90]:
#model.save_weights(path+'/weights/save.h5')

In [79]:
weight_list=glob.glob(path+'/weights/*.h5')
print(weight_list[-1])
model.load_weights(weight_list[-1])

kill_me_baby_datasets/weights\weights_21_0.203_0.939.h5


In [80]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.534051750645493
Test accuracy: 0.9090909090909091


In [81]:
from sklearn.metrics import confusion_matrix

predictions = model.predict(x_test)
print('predictions,shape', predictions.shape)
predict_classes = np.argmax(predictions, 1)
true_classes = np.argmax(y_test, 1)
print(confusion_matrix(true_classes, predict_classes))

predictions,shape (66, 8)
[[ 3  0  1  0  0  0]
 [ 0  1  0  0  0  0]
 [ 1  0  3  1  1  0]
 [ 0  0  0 17  0  0]
 [ 0  0  0  0 34  0]
 [ 1  0  0  0  1  2]]
